In [1]:
import sys
import os

notebook_path = os.getcwd() 
project_root = os.path.dirname(notebook_path)

if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
from copy import deepcopy

In [3]:
import state_NN_models
import Filters
import utils
import Systems
from utils import losses, trainer, utils
from torch.utils.data import TensorDataset, DataLoader, random_split
from state_NN_models.StateBayesianKalmanNet import StateBayesianKalmanNet
from state_NN_models.StateKalmanNet import StateKalmanNet
from state_NN_models.StateKalmanNetWithKnownR import StateKalmanNetWithKnownR

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Používané zařízení: {device}")

Používané zařízení: cuda


In [ ]:
import torch
from math import pi

# =================================================================================
# KROK 1: DEFINICE PARAMETRŮ PRO NELINEÁRNÍ SYSTÉM ("Synthetic")
# =================================================================================

state_dim_nl =1
obs_dim_nl = 1

h_true_nonlinear = lambda x: 0.5 * x
f_true_nonlinear = lambda x: 0.9 * x - 0.05 * x**3 

Q_true = torch.tensor([[0.1]])
R_true = torch.tensor([[0.1]])

Ex0_true = torch.tensor([[1.0]])
P0_true = torch.tensor([[0.5]])

sys_true = Systems.DynamicSystem(state_dim=state_dim_nl,obs_dim=obs_dim_nl,f=f_true_nonlinear,h= h_true_nonlinear,Q= Q_true,R= R_true,Ex0= Ex0_true,P0= P0_true,device=device)

#  Nepřesná dynamika (lineární aproximace nelineární funkce f)
f_model_nonlinear = lambda x: 0.9 * x 
h_model_nonlinear = h_true_nonlinear
# Nepřesná znalost šumu (podcenění Q)
Q_model = torch.tensor([[0.01]])
R_model = torch.tensor([[0.2]])
# Nepřesný počáteční odhad (pro EKF)
Ex0_model = torch.tensor([[0.5]])
P0_model = torch.tensor([[0.5]])

# Sestavení nepřesného modelu pro filtry
# Funkce h, R jsou pro jednoduchost stejné, ale f, Q, Ex0, P0 jsou jiné
sys_model = Systems.DynamicSystem(state_dim=state_dim_nl,obs_dim=obs_dim_nl,f=f_model_nonlinear, h=h_model_nonlinear, Q=Q_model, R=R_model,Ex0= Ex0_model,P0= P0_model,device=device)
# sys_model = Systems.NonlinearSystem(f_true_nonlinear, h_true_nonlinear, Q_true, R_true, Ex0_model, P0_model)


# =================================================================================
# KROK 2: INICIALIZACE OBJEKTŮ SYSTÉMŮ
# =================================================================================
# Ujisti se, že proměnná `device` je definována
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("\nInicializuji 2D 'Synthetic' nelineární systém...")

# Reálný systém, který bude generovat data
# sys_true = DynamicSystem(
#     state_dim=state_dim_nl, obs_dim=obs_dim_nl,
#     Ex0=Ex0_true_nl, P0=P0_true_nl,
#     Q=Q_true_nl, R=R_true_nl,
#     f=f_true_nonlinear, h=h_true_nonlinear, # Předáváme funkce, ne matice
#     device=device
# )

# # Model, který bude používat tvůj KalmanNet (s nepřesnými parametry)
# sys_model = DynamicSystem(
#     state_dim=state_dim_nl, obs_dim=obs_dim_nl,
#     Ex0=Ex0_model_nl, P0=P0_model_nl,
#     Q=Q_model_nl, R=R_model_nl,
#     f=f_model_nonlinear, h=h_model_nonlinear, # Předáváme funkce, ne matice
#     device=device
# )

print("... Nelineární systém inicializován.")


Inicializuji 2D 'Synthetic' nelineární systém...
... Nelineární systém inicializován.


In [6]:
TRAIN_SEQ_LEN = 10      # Krátké sekvence pro stabilní trénink (TBPTT)
VALID_SEQ_LEN = 20      # Stejná délka pro konzistentní validaci
TEST_SEQ_LEN = 100      # Dlouhé sekvence pro testování generalizace

NUM_TRAIN_TRAJ = 500   # Hodně trénovacích příkladů
NUM_VALID_TRAJ = 200    # Dostatek pro spolehlivou validaci
NUM_TEST_TRAJ = 100     # Pro robustní vyhodnocení

BATCH_SIZE = 8         # Dobrý kompromis

x_train, y_train = utils.generate_data(sys_true, num_trajectories=NUM_TRAIN_TRAJ, seq_len=TRAIN_SEQ_LEN)
x_val, y_val = utils.generate_data(sys_true, num_trajectories=NUM_VALID_TRAJ, seq_len=VALID_SEQ_LEN)
x_test, y_test = utils.generate_data(sys_true, num_trajectories=1, seq_len=TEST_SEQ_LEN)

train_dataset = TensorDataset(x_train, y_train)
val_dataset = TensorDataset(x_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [7]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import os
import random
import csv
from datetime import datetime
import pandas as pd
from copy import deepcopy

model_config = {
    "hidden_size_multiplier": 10,
    "output_layer_multiplier": 4,
    "num_gru_layers": 1,
    "init_min_dropout": 0.5,
    "init_max_dropout": 0.8
}

train_config = {
    "total_train_iter": 1200,
    "learning_rate": 1e-4,
    "clip_grad": 10.0,
    "J_samples": 20,
    "validation_period": 20,
    "logging_period": 20,
    "warmup_iterations":0 # Trénuj prvních 400 iterací jen na MSE
}

# =================================================================================
# KROK 3: SPUŠTĚNÍ JEDNOHO TRÉNINKOVÉHO BĚHU
# =================================================================================

print("="*80)
print("Spouštím jeden plnohodnotný tréninkový běh...")
print(f"Parametry modelu: {model_config}")
print(f"Parametry tréninku: {train_config}")
print("="*80)

# Nastavení seedu pro reprodukovatelnost tohoto běhu
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Vytvoření modelu
state_bkn_knet = StateBayesianKalmanNet(
    sys_model,
    device=device,
    **model_config
).to(device)

# Spuštění tréninku
# Používáme `run_training_session`, která vrací slovník s výsledky
results = trainer.training_session_trajectory_with_gaussian_nll_training_fcn(model=state_bkn_knet,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device,
    **train_config
)

# `run_training_session` automaticky načte nejlepší model zpět,
# takže `state_bkn_knet` nyní obsahuje váhy nejlepšího modelu.
trained_model = results['final_model']

print("\n" + "="*80)
print("TRÉNINK DOKONČEN - FINÁLNÍ VÝSLEDKY Z NEJLEPŠÍHO MODELU")
print("="*80)
print(f"Nejlepší model byl nalezen v iteraci: {results['best_iter']}")
# --- Změněné klíče, aby odpovídaly return statementu ---
print(f"Nejlepší dosažený validační ANEES: {results['best_val_anees']:.4f}")
print("--- Metriky odpovídající tomuto nejlepšímu modelu ---")
print(f"  MSE na validační sadě:       {results['best_val_mse']:.4f}")
print(f"  NLL na validační sadě:       {results['best_val_nll']:.4f}")
print("="*80)

# Nyní můžeš s `trained_model` pokračovat, například ho vyhodnotit na testovací sadě.

Spouštím jeden plnohodnotný tréninkový běh...
Parametry modelu: {'hidden_size_multiplier': 10, 'output_layer_multiplier': 4, 'num_gru_layers': 1, 'init_min_dropout': 0.5, 'init_max_dropout': 0.8}
Parametry tréninku: {'total_train_iter': 1200, 'learning_rate': 0.0001, 'clip_grad': 10.0, 'J_samples': 20, 'validation_period': 20, 'logging_period': 20, 'warmup_iterations': 0}
--- Iteration [20/1200] ---
    - Total Loss: 8.1062
    - NLL: 8.1035
    - Reg: 0.0026
    - p1=0.602, p2=0.655

--- Validace v iteraci 20 ---
  Průměrný MSE: 0.8004, Průměrný ANEES: 17.8826
  >>> Nové nejlepší VALIDAČNÍ ANEES! Ukládám model. <<<
--------------------------------------------------
--- Iteration [40/1200] ---
    - Total Loss: 5.8897
    - NLL: 5.8871
    - Reg: 0.0026
    - p1=0.602, p2=0.656

--- Validace v iteraci 40 ---
  Průměrný MSE: 0.8374, Průměrný ANEES: 14.6330
  >>> Nové nejlepší VALIDAČNÍ ANEES! Ukládám model. <<<
--------------------------------------------------
--- Iteration [60/1200] -

In [9]:

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import os
import random
import csv
from datetime import datetime
import pandas as pd
from copy import deepcopy
# Nastavení seedu pro reprodukovatelnost tohoto běhu
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
state_knet = StateKalmanNet(sys_model, device=device, hidden_size_multiplier=12).to(device)
trainer.train_state_KalmanNet(
    model=state_knet, 
    train_loader=train_loader, 
    val_loader=val_loader, 
    device=device, 
    epochs=150, 
    lr=1e-4,
    early_stopping_patience=30
)

Epoch [5/150], Train Loss: 0.740718, Val Loss: 0.747249
Epoch [10/150], Train Loss: 0.725141, Val Loss: 0.734636
Epoch [15/150], Train Loss: 0.721858, Val Loss: 0.731247
Epoch [20/150], Train Loss: 0.711773, Val Loss: 0.719109
Epoch [25/150], Train Loss: 0.706049, Val Loss: 0.766088
Epoch [30/150], Train Loss: 0.704195, Val Loss: 0.720668
Epoch [35/150], Train Loss: 0.700991, Val Loss: 0.717892
Epoch [40/150], Train Loss: 0.696724, Val Loss: 0.721786
Epoch [45/150], Train Loss: 0.697039, Val Loss: 0.720629

Early stopping spuštěno po 49 epochách.
Trénování dokončeno.
Načítám nejlepší model s validační chybou: 0.716527


StateKalmanNet(
  (dnn): DNN_KalmanNet(
    (input_layer): Linear(in_features=4, out_features=96, bias=True)
    (gru): GRU(96, 96)
    (output_layer): Linear(in_features=96, out_features=4, bias=True)
  )
)

In [10]:
import torch
import torch.nn.functional as F
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

# ==============================================================================
# 0. PŘEDPOKLADY
# ==============================================================================
# Předpokládá se, že v proměnných níže máte již natrénované modely:
# trained_model_bkn      <-- Váš BayesianKalmanNet (z dvoufázového tréninku)
# trained_model_classic  <-- Váš StateKalmanNet (z tréninku čistě na MSE)
#
# Přejmenujme je pro srozumitelnost v tomto skriptu:
trained_model_bkn = trained_model # Upravte podle názvu vaší proměnné
trained_model_classic = state_knet       # Upravte podle názvu vaší proměnné


# ==============================================================================
# 1. KONFIGURACE TESTU
# ==============================================================================
TEST_SEQ_LEN = 300
NUM_TEST_TRAJ = 20
J_SAMPLES_TEST = 25

# ==============================================================================
# 2. PŘÍPRAVA DAT
# ==============================================================================
print(f"\nGeneruji {NUM_TEST_TRAJ} testovacích trajektorií o délce {TEST_SEQ_LEN}...")
x_test, y_test = utils.generate_data(sys_true, num_trajectories=NUM_TEST_TRAJ, seq_len=TEST_SEQ_LEN)
test_dataset = TensorDataset(x_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
print("Generování dat dokončeno.")

# ==============================================================================
# 3. INICIALIZACE FILTRŮ PRO POROVNÁNÍ
# ==============================================================================
ekf_mismatched = Filters.ExtendedKalmanFilter(sys_model)
ekf_ideal = Filters.ExtendedKalmanFilter(sys_true)
print("Extended Kalman Filtry inicializovány.")

# ==============================================================================
# 4. VYHODNOCOVACÍ SMYČKA
# ==============================================================================
all_x_true_cpu, all_x_hat_bkn_cpu, all_P_hat_bkn_cpu = [], [], []
all_x_hat_classic_knet_cpu = [] # Klasický KNet neprodukuje P
all_x_hat_ekf_mismatched_cpu, all_P_hat_ekf_mismatched_cpu = [], []
all_x_hat_ekf_ideal_cpu, all_P_hat_ekf_ideal_cpu = [], []

print(f"\nVyhodnocuji modely na {NUM_TEST_TRAJ} testovacích trajektoriích...")

# Důležité: Přepneme oba modely do evaluačního režimu
trained_model_bkn.eval() 
trained_model_classic.eval()

with torch.no_grad():
    for i, (x_true_seq_batch, y_test_seq_batch) in enumerate(test_loader):
        y_test_seq_gpu = y_test_seq_batch.squeeze(0).to(device)
        x_true_seq_gpu = x_true_seq_batch.squeeze(0).to(device)
        initial_state = x_true_seq_gpu[0, :].unsqueeze(0)
        
        # --- A. Vyhodnocení Bayesian KalmanNet (Trajectory-wise) ---
        ensemble_trajectories = []
        for j in range(J_SAMPLES_TEST):
            trained_model_bkn.reset(batch_size=1, initial_state=initial_state)
            current_x_hats = []
            for t in range(1, TEST_SEQ_LEN):
                y_t = y_test_seq_gpu[t, :].unsqueeze(0)
                x_filtered_t, _ = trained_model_bkn.step(y_t)
                current_x_hats.append(x_filtered_t)
            ensemble_trajectories.append(torch.cat(current_x_hats, dim=0))

        ensemble = torch.stack(ensemble_trajectories, dim=0)
        predictions_bkn = ensemble.mean(dim=0)
        diff = ensemble - predictions_bkn.unsqueeze(0)
        outer_products = diff.unsqueeze(-1) @ diff.unsqueeze(-2)
        covariances_bkn = outer_products.mean(dim=0)
        
        full_x_hat_bkn = torch.cat([initial_state, predictions_bkn], dim=0)
        P0_val = trained_model_bkn.system_model.P0.to(device)
        full_P_hat_bkn = torch.cat([P0_val.unsqueeze(0), covariances_bkn], dim=0)

        # --- B. Vyhodnocení klasického StateKalmanNet ---
        trained_model_classic.reset(batch_size=1, initial_state=initial_state)
        classic_knet_preds = []
        for t in range(1, TEST_SEQ_LEN):
            y_t = y_test_seq_gpu[t, :].unsqueeze(0)
            x_filtered_t = trained_model_classic.step(y_t)
            classic_knet_preds.append(x_filtered_t)
        
        predictions_classic_knet = torch.cat(classic_knet_preds, dim=0)
        full_x_hat_classic_knet = torch.cat([initial_state, predictions_classic_knet], dim=0)

        # --- C. Vyhodnocení EKF s nepřesným modelem ---
        Ex0_mismatched = sys_model.Ex0.to(device)
        P0_mismatched = sys_model.P0.to(device)
        ekf_mismatched_results = ekf_mismatched.process_sequence(y_test_seq_gpu, Ex0=Ex0_mismatched, P0=P0_mismatched)
        full_x_hat_ekf_mismatched = torch.cat([Ex0_mismatched.reshape(1, -1), ekf_mismatched_results['x_filtered']], dim=0)
        full_P_hat_ekf_mismatched = torch.cat([P0_mismatched.unsqueeze(0), ekf_mismatched_results['P_filtered']], dim=0)

        # --- D. Vyhodnocení EKF s perfektním modelem (Benchmark) ---
        Ex0_ideal = sys_true.Ex0.to(device)
        P0_ideal = sys_true.P0.to(device)
        ekf_ideal_results = ekf_ideal.process_sequence(y_test_seq_gpu, Ex0=Ex0_ideal, P0=P0_ideal)
        full_x_hat_ekf_ideal = torch.cat([Ex0_ideal.reshape(1, -1), ekf_ideal_results['x_filtered']], dim=0)
        full_P_hat_ekf_ideal = torch.cat([P0_ideal.unsqueeze(0), ekf_ideal_results['P_filtered']], dim=0)

        # --- E. Uložení všech výsledků na CPU ---
        min_len = min(full_x_hat_bkn.shape[0], full_x_hat_classic_knet.shape[0], full_x_hat_ekf_mismatched.shape[0], x_true_seq_gpu.shape[0])
        
        all_x_true_cpu.append(x_true_seq_gpu[:min_len].cpu())
        all_x_hat_bkn_cpu.append(full_x_hat_bkn[:min_len].cpu())
        all_P_hat_bkn_cpu.append(full_P_hat_bkn[:min_len].cpu())
        all_x_hat_classic_knet_cpu.append(full_x_hat_classic_knet[:min_len].cpu())
        all_x_hat_ekf_mismatched_cpu.append(full_x_hat_ekf_mismatched[:min_len].cpu())
        all_P_hat_ekf_mismatched_cpu.append(full_P_hat_ekf_mismatched[:min_len].cpu())
        all_x_hat_ekf_ideal_cpu.append(full_x_hat_ekf_ideal[:min_len].cpu())
        all_P_hat_ekf_ideal_cpu.append(full_P_hat_ekf_ideal[:min_len].cpu())

        print(f"Dokončena trajektorie {i + 1}/{NUM_TEST_TRAJ}...")

# ==============================================================================
# 5. FINÁLNÍ VÝPOČET A VÝPIS METRIK
# ==============================================================================
mse_bkn_list, anees_bkn_list = [], []
mse_classic_knet_list = [] # ANEES zde není
mse_ekf_mismatched_list, anees_ekf_mismatched_list = [], []
mse_ekf_ideal_list, anees_ekf_ideal_list = [], []

print("\nPočítám finální metriky pro jednotlivé trajektorie...")

with torch.no_grad():
    for i in range(NUM_TEST_TRAJ):
        x_true = all_x_true_cpu[i]
        
        # Metriky pro Bayesian KalmanNet
        x_hat_bkn = all_x_hat_bkn_cpu[i]
        P_hat_bkn = all_P_hat_bkn_cpu[i]
        mse_bkn_run = F.mse_loss(x_hat_bkn[1:], x_true[1:]).item()
        anees_bkn_run = utils.calculate_anees_vectorized(x_true.unsqueeze(0), x_hat_bkn.unsqueeze(0), P_hat_bkn.unsqueeze(0))
        mse_bkn_list.append(mse_bkn_run)
        anees_bkn_list.append(anees_bkn_run)

        # Metriky pro klasický KalmanNet
        x_hat_classic_knet = all_x_hat_classic_knet_cpu[i]
        mse_classic_knet_run = F.mse_loss(x_hat_classic_knet[1:], x_true[1:]).item()
        mse_classic_knet_list.append(mse_classic_knet_run)
        
        # Metriky pro nepřesný EKF
        x_hat_ekf_mismatched = all_x_hat_ekf_mismatched_cpu[i]
        P_hat_ekf_mismatched = all_P_hat_ekf_mismatched_cpu[i]
        mse_ekf_mismatched_run = F.mse_loss(x_hat_ekf_mismatched[1:], x_true[1:]).item()
        anees_ekf_mismatched_run = utils.calculate_anees_vectorized(x_true.unsqueeze(0), x_hat_ekf_mismatched.unsqueeze(0), P_hat_ekf_mismatched.unsqueeze(0))
        mse_ekf_mismatched_list.append(mse_ekf_mismatched_run)
        anees_ekf_mismatched_list.append(anees_ekf_mismatched_run)

        # Metriky pro ideální EKF
        x_hat_ekf_ideal = all_x_hat_ekf_ideal_cpu[i]
        P_hat_ekf_ideal = all_P_hat_ekf_ideal_cpu[i]
        mse_ekf_ideal_run = F.mse_loss(x_hat_ekf_ideal[1:], x_true[1:]).item()
        anees_ekf_ideal_run = utils.calculate_anees_vectorized(x_true.unsqueeze(0), x_hat_ekf_ideal.unsqueeze(0), P_hat_ekf_ideal.unsqueeze(0))
        mse_ekf_ideal_list.append(mse_ekf_ideal_run)
        anees_ekf_ideal_list.append(anees_ekf_ideal_run)

        print(f"Traj {i+1:2d} | BKN MSE: {mse_bkn_run:.4f} | KNet MSE: {mse_classic_knet_run:.4f} | EKF-Mis MSE: {mse_ekf_mismatched_run:.4f} | EKF-Ideal MSE: {mse_ekf_ideal_run:.4f}")
        print(f"       | BKN ANEES: {anees_bkn_run:.4f} | KNet ANEES:    N/A    | EKF-Mis ANEES: {anees_ekf_mismatched_run:8.4f} | EKF-Ideal ANEES: {anees_ekf_ideal_run:8.4f}")
        print("-" * 110)

# Průměrování výsledků
avg_mse_bkn = np.mean(mse_bkn_list)
avg_anees_bkn = np.mean(anees_bkn_list)
avg_mse_classic_knet = np.mean(mse_classic_knet_list)
avg_mse_ekf_mismatched = np.mean(mse_ekf_mismatched_list)
avg_anees_ekf_mismatched = np.mean(anees_ekf_mismatched_list)
avg_mse_ekf_ideal = np.mean(mse_ekf_ideal_list)
avg_anees_ekf_ideal = np.mean(anees_ekf_ideal_list)
state_dim_for_nees = all_x_true_cpu[0].shape[1]

print("\n" + "="*80)
print(f"FINÁLNÍ VÝSLEDKY (průměr přes {NUM_TEST_TRAJ} běhů)")
print("="*80)
print("\n--- Průměrná MSE (přesnost) ---")
print(f"Bayesian KNet (BKN):              {avg_mse_bkn:.4f}")
print(f"Klasický KNet (pouze MSE):        {avg_mse_classic_knet:.4f} <-- Nejlepší data-driven přesnost")
print(f"EKF (Nepřesný model):             {avg_mse_ekf_mismatched:.4f}")
print(f"EKF (Ideální model - Benchmark):    {avg_mse_ekf_ideal:.4f} <-- Teoretický limit přesnosti")
print("\n--- Průměrný ANEES (kredibilita/kalibrace) ---")
print(f"Očekávaná hodnota:   {state_dim_for_nees:.4f}")
print("--------------------------------------------------")
print(f"Bayesian KNet (BKN):              {avg_anees_bkn:.4f}")
print(f"Klasický KNet (pouze MSE):        N/A (model neprodukuje kovarianci)")
print(f"EKF (Nepřesný model):             {avg_anees_ekf_mismatched:.4f}")
print(f"EKF (Ideální model - Benchmark):    {avg_anees_ekf_ideal:.4f}")
print("="*80)


Generuji 20 testovacích trajektorií o délce 300...
Generování dat dokončeno.
Extended Kalman Filtry inicializovány.

Vyhodnocuji modely na 20 testovacích trajektoriích...
Dokončena trajektorie 1/20...
Dokončena trajektorie 2/20...
Dokončena trajektorie 3/20...
Dokončena trajektorie 4/20...
Dokončena trajektorie 5/20...
Dokončena trajektorie 6/20...
Dokončena trajektorie 7/20...
Dokončena trajektorie 8/20...
Dokončena trajektorie 9/20...
Dokončena trajektorie 10/20...
Dokončena trajektorie 11/20...
Dokončena trajektorie 12/20...
Dokončena trajektorie 13/20...
Dokončena trajektorie 14/20...
Dokončena trajektorie 15/20...
Dokončena trajektorie 16/20...
Dokončena trajektorie 17/20...
Dokončena trajektorie 18/20...
Dokončena trajektorie 19/20...
Dokončena trajektorie 20/20...

Počítám finální metriky pro jednotlivé trajektorie...
Traj  1 | BKN MSE: 0.6563 | KNet MSE: 0.5861 | EKF-Mis MSE: 1.9800 | EKF-Ideal MSE: 2.4244
       | BKN ANEES: 2.1902 | KNet ANEES:    N/A    | EKF-Mis ANEES: 107